### 04 Embedding model（埋め込みモデル）と RAG（検索拡張生成）
gpt-oss:20b＋MCPサーバ構成のため、**Colab GPU は L4 を使用すること。**
- 必要なライブラリをインストール
- Google Colab に Ollama をセットアップ
  - LLM モデルは gpt-oss:20b を使用（Ollama）
  - Embedding モデルは bge-m3 を使用（Ollama）
  - Reranker モデルは BAAI/bge-reranker-v2-m3 を使用（Sentence Transformers）
- data フォルダからデータを読み込み
> [井澤克彦, 市川信一郎, 高速回転ホイール: 高速回転ホイール開発を通しての知見, 宇宙航空研究開発機構研究開発報告, 2008](https://jaxa.repo.nii.ac.jp/records/2149)
- データの前処理
  - markdown に変換（MarkItDown を使用）
  - Unicode正規化 (NFKC)
  - チャンク分割
    - LangChain の SpacyTextSplitter を使用
    - spaCy の日本語モデルは、ja_ginza を使用
- ベクトルデータベースの構築（ChromaDB, インメモリ）
- 検索機能の実装と単体動作確認
　- キーワード検索 @ BM25（spaCyで形態素解析の前処理が必要）
  - Embedding model によるセマンティック検索
  - ハイブリッド検索
  - Reranker による再順位付け
- 検索機能をLLM の tool として定義
- 動作確認

**必要なライブラリをインストール**
- 1行にまとめることで pip が全パッケージの依存関係を一括解決する。
- 分割すると後勝ちで依存関係が壊れるリスクがある。
- NOTE: Colab では uv ではなく pip を使う。
> uv は依存解決の過程で numpy 等をアップグレードし、プリインストール済みの scipy 等を壊すため。

In [ ]:
# Google Colab に必要なライブラリをインストールする。
# 1行にまとめることで pip が全パッケージの依存関係を一括解決する。
# NOTE: Colab では uv ではなく pip を使う。uv は依存解決の過程で
#       numpy 等をアップグレードし、プリインストール済みの scipy 等を壊すため。
# NOTE: langchain 関連は 1.x 系に明示的に指定する。
#       Colab プリインストールの 0.3.x が残ると langchain-mcp-adapters が動作しない。
%pip install -U ollama langchain-ollama \
     "langchain>=1.2.8" "langchain-core>=1.2.8" \
     "langgraph>=1.0.7" \
     markitdown chromadb \
     "langchain-text-splitters>=0.3" \
     spacy ginza ja-ginza \
     rank-bm25 sentence-transformers

**Google Colab に Ollama をセットアップ**
- Ollama のインストール・起動・モデルのダウンロードを行う。
- 詳細は [01_connect_oss_llm.ipynb](01_connect_oss_llm.ipynb) を参照。

In [ ]:
# Ollama のインストール・起動・モデルのダウンロード
# 詳細は 01_connect_oss_llm.ipynb を参照
import subprocess
import time
import ollama  # type: ignore

!apt-get install -y -qq zstd
!curl -fsSL https://ollama.com/install.sh | sh

process = subprocess.Popen(
    ["ollama", "serve"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL,
)
time.sleep(5)


def ollama_pull(model: str) -> None:
    """Ollama モデルをダウンロードし、進捗をインライン表示する。

    NOTE: ollama pull のプログレスバーは Colab で文字化けするため、
          Python API 経由でステータスのみ表示する。
    """
    for progress in ollama.pull(model, stream=True):
        status = progress.get("status", "")
        total = progress.get("total") or 0
        completed = progress.get("completed") or 0
        if total:
            line = f"{status}: {completed / total:.0%}"
        else:
            line = status
        print(f"\r{line:<60}", end="", flush=True)
    print(f"\n{model}: Done!")


# AI エージェントにはツールコール対応モデルが必要。
model_name = "gpt-oss:20b"
ollama_pull(model_name)
!ollama show {model_name}

**ChatOllama で LLM に接続**
- 詳細は [01_connect_oss_llm.ipynb](01_connect_oss_llm.ipynb) を参照。

In [3]:
# ChatOllama で LLM に接続する。
from langchain_ollama import ChatOllama  # type: ignore

llm = ChatOllama(
    model="gpt-oss:20b",
    num_ctx=16384,
    num_predict=-1,
    temperature=0.8,
    top_k=40,
    top_p=0.9,
    repeat_penalty=1.1,
    reasoning=None,
)

**Embedding モデル（bge-m3）をダウンロード**
- bge-m3 は多言語対応の Embedding モデル。日本語にも対応。
- Ollama 経由で利用する。

In [ ]:
# Embedding モデル (bge-m3) をダウンロードする。
embedding_model_name = "bge-m3"
ollama_pull(embedding_model_name)
!ollama show {embedding_model_name}

**OllamaEmbeddings と Reranker モデルのセットアップ**
- OllamaEmbeddings: LangChain 経由で bge-m3 を Embedding に使用。
- Reranker: Sentence Transformers の CrossEncoder で BAAI/bge-reranker-v2-m3 を使用。

In [ ]:
# OllamaEmbeddings: LangChain 経由で bge-m3 を Embedding に使用する。
from langchain_ollama import OllamaEmbeddings  # type: ignore

embeddings = OllamaEmbeddings(model=embedding_model_name)

# 動作確認: 短いテキストを埋め込んでベクトル次元を確認する。
test_vec = embeddings.embed_query("テスト文です")
print(f"Embedding dim: {len(test_vec)}")

In [ ]:
# Reranker: Sentence Transformers の CrossEncoder を使用する。
# 初回実行時にモデルがダウンロードされる。
from sentence_transformers import CrossEncoder  # type: ignore

reranker = CrossEncoder("BAAI/bge-reranker-v2-m3")
print("Reranker model loaded.")

**data フォルダからデータを読み込み**

> [井澤克彦, 市川信一郎, 高速回転ホイール: 高速回転ホイール開発を通しての知見, 宇宙航空研究開発機構研究開発報告, 2008](https://jaxa.repo.nii.ac.jp/records/2149)

In [ ]:
# data フォルダから PDF を読み込み、MarkItDown で markdown に変換する。
from pathlib import Path
from markitdown import MarkItDown  # type: ignore

data_dir = Path("data")
pdf_path = data_dir / "高速回転ホイール_高速回転ホイール開発を通しての知見.pdf"

md = MarkItDown()
result = md.convert(str(pdf_path))
raw_text = result.text_content

print(f"文字数: {len(raw_text)}")
print("--- 先頭 500 文字 ---")
print(raw_text[:500])

**データの前処理**
- Unicode 正規化 (NFKC)
- チャンク分割（SpacyTextSplitter + ja_ginza）

In [ ]:
# Unicode 正規化 (NFKC) を適用する。
# 全角英数→半角、半角カナ→全角 などを統一する。
import unicodedata

text = unicodedata.normalize("NFKC", raw_text)
print(f"正規化後の文字数: {len(text)}")

In [ ]:
# SpacyTextSplitter でチャンク分割する。
# spaCy の日本語モデル ja_ginza を使用し、文境界を考慮して分割する。
from langchain_text_splitters import SpacyTextSplitter  # type: ignore

CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200

text_splitter = SpacyTextSplitter(
    pipeline="ja_ginza",
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
)

chunks = text_splitter.split_text(text)

print(f"チャンク数: {len(chunks)}")
for i, chunk in enumerate(chunks[:3]):
    print(f"\n--- Chunk {i} ({len(chunk)} chars) ---")
    print(chunk[:200] + "..." if len(chunk) > 200 else chunk)

**ベクトルデータベースの構築（ChromaDB, インメモリ）**
- チャンクを Embedding してベクトルデータベースに格納する。

In [ ]:
# ChromaDB にチャンクを格納する（インメモリ）。
import chromadb  # type: ignore

chroma_client = chromadb.Client()

collection = chroma_client.create_collection(
    name="jaxa_wheel",
    metadata={"hnsw:space": "cosine"},
)

# チャンクを Embedding してデータベースに追加する。
chunk_embeddings = embeddings.embed_documents(chunks)

collection.add(
    ids=[f"chunk_{i}" for i in range(len(chunks))],
    documents=chunks,
    embeddings=chunk_embeddings,
)

print(f"ChromaDB に {collection.count()} 件のチャンクを格納しました。")

**検索機能の実装**
- キーワード検索 @ BM25（spaCy で形態素解析の前処理が必要）
- Embedding model によるセマンティック検索
- ハイブリッド検索
- Reranker による再順位付け

In [ ]:
# BM25 用の前処理: spaCy (ja_ginza) で形態素解析してトークン化する。
import spacy  # type: ignore
from rank_bm25 import BM25Okapi  # type: ignore

nlp = spacy.load("ja_ginza")

TOP_K = 5


def tokenize(text: str) -> list[str]:
    """spaCy で形態素解析し、名詞・動詞・形容詞のレンマを返す。"""
    doc = nlp(text)
    return [
        token.lemma_
        for token in doc
        if token.pos_ in ("NOUN", "VERB", "ADJ", "PROPN") and len(token.lemma_) > 1
    ]


# チャンクをトークン化して BM25 インデックスを構築する。
tokenized_chunks = [tokenize(chunk) for chunk in chunks]
bm25 = BM25Okapi(tokenized_chunks)

print(f"BM25 インデックス構築完了: {len(tokenized_chunks)} 件")
print(f"トークン例（先頭チャンク）: {tokenized_chunks[0][:10]}")

In [ ]:
import numpy as np

RETRIEVAL_TOP_K = 20  # 第1段検索（BM25 / セマンティック / ハイブリッド）の抽出数
RERANK_TOP_K = 5  # 第2段検索（Reranker）の抽出数


def search_bm25(query: str, top_k: int = RETRIEVAL_TOP_K) -> list[dict]:
    """BM25 によるキーワード検索を行う。"""
    tokenized_query = tokenize(query)
    scores = bm25.get_scores(tokenized_query)
    top_indices = np.argsort(scores)[::-1][:top_k]
    return [
        {
            "rank": rank + 1,
            "chunk_id": int(idx),
            "score": float(scores[idx]),
            "text": chunks[idx],
        }
        for rank, idx in enumerate(top_indices)
        if scores[idx] > 0
    ]


def search_semantic(query: str, top_k: int = RETRIEVAL_TOP_K) -> list[dict]:
    """Embedding model によるセマンティック検索を行う。"""
    query_embedding = embeddings.embed_query(query)
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k,
    )
    return [
        {
            "rank": rank + 1,
            "chunk_id": int(doc_id.split("_")[1]),
            "score": 1.0 - dist,
            "text": doc,
        }
        for rank, (doc_id, doc, dist) in enumerate(
            zip(results["ids"][0], results["documents"][0], results["distances"][0])
        )
    ]


def search_hybrid(
    query: str, top_k: int = RETRIEVAL_TOP_K, bm25_weight: float = 0.3
) -> list[dict]:
    """BM25 とセマンティック検索のハイブリッド検索を行う。

    Reciprocal Rank Fusion (RRF) でスコアを統合する。
    """
    k = 60  # RRF のハイパーパラメータ

    bm25_results = search_bm25(query, top_k=top_k)
    semantic_results = search_semantic(query, top_k=top_k)

    # RRF スコアを計算する。
    scores: dict[int, float] = {}
    texts: dict[int, str] = {}

    for r in bm25_results:
        cid = r["chunk_id"]
        scores[cid] = scores.get(cid, 0) + bm25_weight / (k + r["rank"])
        texts[cid] = r["text"]

    semantic_weight = 1.0 - bm25_weight
    for r in semantic_results:
        cid = r["chunk_id"]
        scores[cid] = scores.get(cid, 0) + semantic_weight / (k + r["rank"])
        texts[cid] = r["text"]

    sorted_ids = sorted(scores, key=lambda cid: scores[cid], reverse=True)[:top_k]
    return [
        {"rank": rank + 1, "chunk_id": cid, "score": scores[cid], "text": texts[cid]}
        for rank, cid in enumerate(sorted_ids)
    ]


def rerank(query: str, results: list[dict], top_k: int = RERANK_TOP_K) -> list[dict]:
    """Reranker (CrossEncoder) で検索結果を再順位付けする。"""
    if not results:
        return []
    pairs = [(query, r["text"]) for r in results]
    scores = reranker.predict(pairs)
    ranked_indices = np.argsort(scores)[::-1][:top_k]
    return [
        {
            "rank": rank + 1,
            "chunk_id": results[idx]["chunk_id"],
            "score": float(scores[idx]),
            "text": results[idx]["text"],
        }
        for rank, idx in enumerate(ranked_indices)
    ]


print("検索関数を定義しました: search_bm25, search_semantic, search_hybrid, rerank")
print(f"  第1段検索 top_k: {RETRIEVAL_TOP_K}, 第2段 Rerank top_k: {RERANK_TOP_K}")

**検索機能の単体動作確認**

In [ ]:
# 検索機能の単体動作確認
test_query = "高速回転ホイールの寿命試験"

print("=" * 60)
print(f"テストクエリ: {test_query}")
print("=" * 60)

# 1. BM25 キーワード検索
print("\n--- BM25 キーワード検索 ---")
bm25_results = search_bm25(test_query)
for r in bm25_results[:3]:
    print(f"  Rank {r['rank']} (score={r['score']:.4f}, chunk_id={r['chunk_id']})")
    print(f"    {r['text'][:100]}...")

# 2. セマンティック検索
print("\n--- セマンティック検索 ---")
sem_results = search_semantic(test_query)
for r in sem_results[:3]:
    print(f"  Rank {r['rank']} (score={r['score']:.4f}, chunk_id={r['chunk_id']})")
    print(f"    {r['text'][:100]}...")

# 3. ハイブリッド検索
print("\n--- ハイブリッド検索 ---")
hybrid_results = search_hybrid(test_query)
for r in hybrid_results[:3]:
    print(f"  Rank {r['rank']} (score={r['score']:.6f}, chunk_id={r['chunk_id']})")
    print(f"    {r['text'][:100]}...")

# 4. Reranker による再順位付け（ハイブリッド検索結果を入力）
print("\n--- Reranker 再順位付け ---")
reranked_results = rerank(test_query, hybrid_results)
for r in reranked_results:
    print(f"  Rank {r['rank']} (score={r['score']:.4f}, chunk_id={r['chunk_id']})")
    print(f"    {r['text'][:100]}...")

**検索機能を LLM の tool として定義**
- ハイブリッド検索 + Reranker を LangChain の `@tool` デコレータで定義する。
- LLM がユーザの質問に対して自動的に検索ツールを呼び出し、RAG を実現する。

In [ ]:
# 検索機能を LLM の tool として定義する。
from langchain_core.tools import tool  # type: ignore


@tool
def search_document(query: str) -> str:
    """高速回転ホイールに関する技術文書を検索します。
    ハイブリッド検索（BM25 + セマンティック検索）と Reranker を組み合わせて、
    クエリに最も関連するテキストを返します。

    Args:
        query: 検索クエリ（日本語）
    """
    hybrid_results = search_hybrid(query)
    reranked = rerank(query, hybrid_results)

    if not reranked:
        return "検索結果が見つかりませんでした。"

    passages = []
    for r in reranked:
        passages.append(
            f"[チャンク {r['chunk_id']}] (スコア: {r['score']:.4f})\n{r['text']}"
        )
    return "\n\n---\n\n".join(passages)


tools = [search_document]

print("=== RAG Tools ===")
for t in tools:
    print(f"  - {t.name}: {t.description[:80]}...")

**動作確認**
- ReAct エージェントで検索ツールを使った RAG の動作を確認する。

In [ ]:
# ReAct エージェントを構築して RAG の動作確認を行う。
from langchain.agents import create_agent  # type: ignore
from langgraph.checkpoint.memory import InMemorySaver  # type: ignore

memory = InMemorySaver()

system_prompt = """
あなたは高速回転ホイールに関する技術文書の専門家AIアシスタントです。
ユーザの質問に日本語で回答してください。
必ず search_document ツールで文書を検索し、検索結果に基づいて回答してください。
検索結果に含まれない内容は回答しないでください。

回答の最後に、以下の形式で結論をまとめてください。

# 結論
- ユーザの質問: （質問内容）
- 回答: （簡潔な回答）
"""

agent = create_agent(
    model=llm,
    tools=tools,
    checkpointer=memory,
    system_prompt=system_prompt,
)

In [ ]:
# 動作確認: エージェントに質問する。
config = {"configurable": {"thread_id": "rag-test"}}

response = await agent.ainvoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "高速回転ホイールの寿命試験ではどのような結果が得られましたか？",
            }
        ]
    },
    config=config,
)

print("=== RAG Agent Result ===\n")
for msg in response["messages"]:
    if isinstance(msg.content, list):
        msg.content = "\n".join(
            item["text"] for item in msg.content if item.get("type") == "text"
        )
    msg.pretty_print()